In [ ]:
!uv pip install -U transformers accelerate tiktoken

In [2]:
# Load Qwen model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-0.6B"
device = ("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    trust_remote_code=True # False to disable loading custom model code from the Hub (for security reasons), loading local model only if available
)
print(f"✅ Model {model_name} loaded")
print(f"Vocabulary size: {tokenizer.vocab_size:,}")
print(f"Model parameters: ~{sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

/home/admin/_github/massimodipaolo/ai-crash-course/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


✅ Model Qwen/Qwen3-0.6B loaded
Vocabulary size: 151,643
Model parameters: ~596.0M


In [13]:
# Helper function to visualize tokenization
def show_tokens(text, tokenizer):
    """Visualize how text is tokenized"""
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    
    print(f"Original text: {text}, tokenized into {len(tokens)} tokens.")
    print(f"\nTokens: {tokens}")
    print(f"Token IDs: {token_ids}")
def generate_response(model, tokenizer, messages):
    # Apply the chat template
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, # we will tokenize later
        add_generation_prompt=True,
        enable_thinking=False
    )
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate the output
    outputs = model.generate(
        **inputs,
        max_new_tokens=255,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the output, skipping special tokens and the prompt itself
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()           

### Why Can't LLMs Spell Words?

LLMs see tokens, not characters. When you ask "How many 'r's in 'strawberry'?", the model doesn't see individual letters - it sees subword tokens. The word might be split as `['straw', 'berry']`, making it impossible to count letters accurately.

In [52]:
# Example: Counting letters in "strawberry"
from tokenizers import Tokenizer
import torch
word = "strawberry"
print(f"Question: How many 'r's are in '{word}'?")
print(f"Correct answer: {word.count('r')}\n")

# Show how the tokenizer sees it
show_tokens(word, tokenizer)

print("\n" + "="*60)
print("The model sees tokens, not individual characters!")
print("It can't count letters it can't see individually.")
print("\n" + "="*60)

# Let's test with a sentence
prompt = "How many 'r's are in the sentence: 'The strawberry is red'?"
outputs = generate_response(model, tokenizer, [{"role":"user","content":prompt}])
print(outputs)

print("\n" + "-"*60)
prompt = "How many 'berry's are in the sentence: 'The strawberry is red'?"
outputs = generate_response(model, tokenizer, [{"role":"user","content":prompt}])
print(outputs)

Question: How many 'r's are in 'strawberry'?
Correct answer: 3

Original text: strawberry, tokenized into 3 tokens.

Tokens: ['str', 'aw', 'berry']
Token IDs: [495, 672, 15357]

The model sees tokens, not individual characters!
It can't count letters it can't see individually.

The sentence is: **"The strawberry is red"**

Breaking it down word by word:

- "The"  
- "strawberry"  
- "is"  
- "red"

There are **two 'r's**: one in "strawberry" and one in "red".  

So, the answer is: **2**.

------------------------------------------------------------
The sentence is: **"The strawberry is red"**

Breaking it down word by word:

- "The"  
- "strawberry"  
- "is"  
- "red"

There are **two 'r's**: one in "strawberry" and one in "red".  

So, the answer is: **2**.

------------------------------------------------------------
The sentence "The strawberry is red" contains **1** berry.
The sentence "The strawberry is red" contains **1** berry.


### Why Can't LLMs Reverse Strings?

String reversal requires character-level operations. But tokens are not characters! If "hello" is one token, the model has no way to access individual characters to reverse them. Token boundaries make character-level manipulation nearly impossible.

In [53]:
# Example: Reversing strings
test_strings = ["singularity", "world", "tokenization", "crashing"]

for text in test_strings:
    show_tokens(text, tokenizer)
for text in test_strings:
    print("\n" + "-"*60)
    prompt = f"Reverse this word: '{text}'?"
    outputs = generate_response(model, tokenizer, [{"role":"user","content":prompt}])
    print(f"{text}[={text[::-1]}] {outputs} {'✅' if text[::-1] in outputs else '❌'}")

Original text: singularity, tokenized into 2 tokens.

Tokens: ['sing', 'ularity']
Token IDs: [79307, 28979]
Original text: world, tokenized into 1 tokens.

Tokens: ['world']
Token IDs: [14615]
Original text: tokenization, tokenized into 2 tokens.

Tokens: ['token', 'ization']
Token IDs: [5839, 2022]
Original text: crashing, tokenized into 2 tokens.

Tokens: ['cr', 'ashing']
Token IDs: [5082, 19049]

------------------------------------------------------------
singularity[=ytiralugnis] To reverse the word **"singularity"**, we start by writing it in reverse order:

**"gnitsiligns"**. ❌

------------------------------------------------------------
singularity[=ytiralugnis] To reverse the word **"singularity"**, we start by writing it in reverse order:

**"gnitsiligns"**. ❌

------------------------------------------------------------
world[=dlrow] The reversed word is **"drow"**. ❌

------------------------------------------------------------
world[=dlrow] The reversed word is **"drow"**

### Why Are LLMs Worse at Non-English Languages?

Tokenizers are typically trained on datasets with predominantly English text. This means:
- English words often get single tokens
- Non-English text (like Japanese, Arabic, etc.) gets fragmented into many more tokens
- More tokens = longer sequences = more computation = worse performance
- The model sees less "context" for the same amount of text

In [56]:
# Compare tokenization efficiency: English vs other languages
sentences = {
    "English": "Hello, how are you today?",
    "Spanish": "Hola, ¿cómo estás hoy?",
    "Italian": "Ciao, come stai oggi?",
    "French": "Bonjour, comment allez-vous aujourd'hui?",
    "German": "Hallo, wie geht es dir heute?",
    "Japanese": "こんにちは、今日はお元気ですか？",
    "Arabic": "مرحبا، كيف حالك اليوم؟",
    "Chinese": "你好，你今天好吗？",
    "Korean": "안녕하세요, 오늘 어떻게 지내세요?",
}

print("Token efficiency comparison:\n")
print(f"{'Language':<12} {'Characters':<12} {'Tokens':<8} {'Chars/Token':<12} {'Efficiency'}")
print("=" * 70)

english_ratio = None
for lang, text in sentences.items():
    char_count = len(text)
    tokens = tokenizer.tokenize(text)
    token_count = len(tokens)
    ratio = char_count / token_count if token_count > 0 else 0
    
    if lang == "English":
        english_ratio = ratio
        efficiency = "baseline"
    else:
        efficiency = f"{(ratio / english_ratio * 100):.0f}%" if english_ratio else "N/A"
    
    print(f"{lang:<12} {char_count:<12} {token_count:<8} {ratio:<12.2f} {efficiency}")
    
print("\n💡 Lower chars/token = more tokens needed = worse performance!")

Token efficiency comparison:

Language     Characters   Tokens   Chars/Token  Efficiency
English      25           7        3.57         baseline
Spanish      22           9        2.44         68%
Italian      21           8        2.62         74%
Pesarese     23           9        2.56         72%
French       40           9        4.44         124%
German       29           8        3.62         101%
Japanese     16           8        2.00         56%
Arabic       22           9        2.44         68%
Chinese      9            6        1.50         42%
Korean       19           10       1.90         53%

💡 Lower chars/token = more tokens needed = worse performance!


In [58]:
# Deep dive: Japanese tokenization
japanese_text = "こんにちは、今日はお元気ですか？"
print(f"Japanese text: {japanese_text}")
print(f"Character count: {len(japanese_text)}")

tokens = tokenizer.tokenize(japanese_text)
show_tokens(japanese_text, tokenizer)

print("\n" + "="*60)
print("Each character often becomes its own token or gets fragmented!")
print("This makes Japanese text ~3-4x more 'expensive' than English.")

Japanese text: こんにちは、今日はお元気ですか？
Character count: 16
Original text: こんにちは、今日はお元気ですか？, tokenized into 8 tokens.

Tokens: ['ãģĵãĤĵãģ«ãģ¡ãģ¯', 'ãĢģ', 'ä»ĬæĹ¥ãģ¯', 'ãģĬ', 'åħĥ', 'æ°Ĺ', 'ãģ§ãģĻãģĭ', 'ï¼Ł']
Token IDs: [89015, 5373, 133165, 32234, 23305, 94121, 131938, 11319]

Each character often becomes its own token or gets fragmented!
This makes Japanese text ~3-4x more 'expensive' than English.


### Why Are LLMs Bad at Simple Arithmetic?

Numbers are tokenized inconsistently! Sometimes "123" is one token, sometimes three tokens "1", "2", "3". Sometimes "1234" becomes "12" + "34". This inconsistency makes it impossible for the model to learn reliable arithmetic patterns.

Multi-digit numbers, decimal points, and number formatting all affect tokenization, making math operations unpredictable.

In [18]:
# number tokenization is consistent?
numbers = ["1", "12", "12 cows", "123", "365", "1234", "1984","2001", "2037",
           "3.14", "1,000", "1000", "1_000"]

print("Number tokenization inconsistencies?:\n")
print(f"{'Number':<12} {'Tokens':<8} {'Token Breakdown'}")
print("=" * 60)

for num in numbers:
    tokens = tokenizer.tokenize(num)
    token_count = len(tokens)
    print(f"{num:<12} {token_count:<8} {tokens}")

Number tokenization inconsistencies?:

Number       Tokens   Token Breakdown
1            1        ['1']
12           2        ['1', '2']
12 cows      3        ['1', '2', 'Ġcows']
123          3        ['1', '2', '3']
365          3        ['3', '6', '5']
1234         4        ['1', '2', '3', '4']
1984         4        ['1', '9', '8', '4']
2001         4        ['2', '0', '0', '1']
2037         4        ['2', '0', '3', '7']
3.14         4        ['3', '.', '1', '4']
1,000        5        ['1', ',', '0', '0', '0']
1000         4        ['1', '0', '0', '0']
1_000        5        ['1', '_', '0', '0', '0']


In [20]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

print("\nBERT WordPiece Number Tokenization:\n")
print(f"{'Number':<12} {'Tokens':<8} {'Token Breakdown'}")
print("=" * 60)

for num in numbers:
    tokens = tokenizer_bert.tokenize(num)
    token_count = len(tokens)
    print(f"{num:<12} {token_count:<8} {tokens}")

print("\n💡 BERT splits numbers inconsistently with '##' prefix for continuations, and...")


BERT WordPiece Number Tokenization:

Number       Tokens   Token Breakdown
1            1        ['1']
12           1        ['12']
12 cows      2        ['12', 'cows']
123          1        ['123']
365          1        ['365']
1234         2        ['123', '##4']
1984         1        ['1984']
2001         1        ['2001']
2037         2        ['203', '##7']
3.14         3        ['3', '.', '14']
1,000        3        ['1', ',', '000']
1000         1        ['1000']
1_000        3        ['1', '_', '000']

💡 BERT splits numbers inconsistently with '##' prefix for continuations, and...


In [26]:
import tiktoken

# gpt-2 tokenizer
tokenizer_gpt2 = tiktoken.encoding_for_model("gpt-2")
numbers = ["1", "12", "12 cows", "123", "365", "1234", "1984","2001", "2037",
           "3.14", "1,000", "1000", "1_000"]
print("(tiktoken) Number Tokenization:\n")
print(f"{'Number':<12} {'Tokens':<8} {'Token IDs'}")
print("=" * 60)

for num in numbers:
    tokens = tokenizer_gpt2.encode(num)
    decoded_tokens = [tokenizer_gpt2.decode([t]) for t in tokens]
    print(f"{num:<12} {len(tokens):<8} {decoded_tokens}")

print("\n💡 Notice: '1234' vs '1984' tokenize differently!")
print("'1984' is often one token (famous year with Orwell works literature), same for '2001' (2001: A Space Odyssey)")
print("'1234' might be two tokens")

(tiktoken) Number Tokenization:

Number       Tokens   Token IDs
1            1        ['1']
12           1        ['12']
12 cows      2        ['12', ' cows']
123          1        ['123']
365          1        ['365']
1234         2        ['12', '34']
1984         1        ['1984']
2001         1        ['2001']
2037         2        ['20', '37']
3.14         3        ['3', '.', '14']
1,000        3        ['1', ',', '000']
1000         1        ['1000']
1_000        3        ['1', '_', '000']

💡 Notice: '1234' vs '1984' tokenize differently!
'1984' is often one token (famous year with Orwell works literature), same for '2001' (2001: A Space Odyssey)
'1234' might be two tokens


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("../.env")
from transformers import AutoTokenizer
from huggingface_hub import login
login(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

# LLaMA tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

print("\nLLaMA-x (SentencePiece) Number Tokenization:\n")
print(f"{'Number':<12} {'Tokens':<8} {'Token Breakdown'}")
print("=" * 60)

for num in numbers:
    tokens = tokenizer_llama.tokenize(num)
    token_count = len(tokens)
    print(f"{num:<12} {token_count:<8} {tokens}")

print("\n💡 SentencePiece uses '▁' for word boundaries, numbers still inconsistent")

In [38]:
from transformers import pipeline
# Demonstrate the arithmetic problem with specific examples
arithmetic_problems = [
    "1+1",
    "1204543 + 4545456",
    "788799 - 23545434",
    "0.1812 * 7734",
    "100 // 5"
]

print("How tokenization affects arithmetic expressions:\n")
for problem in arithmetic_problems:
    tokens = tokenizer.tokenize(problem)
    print(f"Expression: {problem}, tokenized into {len(tokens)} tokens. Evaluation: {eval(problem)}")

print("="*60)
print("The model doesn't see '123' as one hundred twenty-three.")
print("It sees it as whatever tokens the tokenizer produces!")
print("Digit-level understanding is lost in tokenization.")

text_generation = pipeline("text-generation", model=model, tokenizer=tokenizer)
for problem in arithmetic_problems:
    prompt = f"Solve this arithmetic problem, output only the result, not the explanation: {problem}\nAnswer:"
    outputs = text_generation(prompt, max_new_tokens=50)
    answer = outputs[0]['generated_text']
    print(f"Prompt: {prompt}")
    print(f"Model Output: {answer}\n{"✅" if str(eval(problem)) in answer.strip() else "❌"}")

Device set to use cuda


How tokenization affects arithmetic expressions:

Expression: 1+1, tokenized into 3 tokens. Evaluation: 2
Expression: 1204543 + 4545456, tokenized into 16 tokens. Evaluation: 5749999
Expression: 788799 - 23545434, tokenized into 16 tokens. Evaluation: -22756635
Expression: 0.1812 * 7734, tokenized into 12 tokens. Evaluation: 1401.4008
Expression: 100 // 5, tokenized into 6 tokens. Evaluation: 20
The model doesn't see '123' as one hundred twenty-three.
It sees it as whatever tokens the tokenizer produces!
Digit-level understanding is lost in tokenization.
Prompt: Solve this arithmetic problem, output only the result, not the explanation: 1+1
Answer:
Model Output: Solve this arithmetic problem, output only the result, not the explanation: 1+1
Answer: 2

The problem is 1+1, which is 2. The result is 2.
The answer is 2.
The result is 2.
The answer is 2.
The result is 2.

The answer is 
✅
Prompt: Solve this arithmetic problem, output only the result, not the explanation: 1204543 + 4545456
A

### Glitch Tokens: The "SolidGoldMagikarp" Problem

Some tokens appear in the vocabulary but were rarely or never seen during training. These "glitch tokens" can cause bizarre behavior because:
- The model has no learned representation for them
- They might trigger unpredictable outputs
- They expose the gap between vocabulary and training data

Famous example: "SolidGoldMagikarp" was a Reddit username that became a token in GPT models but caused strange behaviors because it was never properly trained. E.g. "What does the word 'SolidGoldMagikarp' mean?" -> "The word 'distribute' refers to ..." 

In [8]:
import numpy as np
from collections import Counter

# Get the vocabulary
vocab = tokenizer.get_vocab()
vocab_items = list(vocab.items())
print(f"Total vocabulary size: {len(vocab):,}\n")

# Find tokens with unusual characteristics
unusual_tokens = []

for token, token_id in vocab_items:
    # Skip control tokens
    if token_id < 100:  # Usually special tokens
        continue
    
    # Decode the token to see what it represents
    try:
        decoded = tokenizer.decode([token_id])
        
        # Look for unusual patterns:
        # 1. Very long tokens (might be rare combinations)
        if len(token) > 15:
            unusual_tokens.append(("long", token, token_id, decoded))
        
        # 2. Tokens with lots of special characters
        special_char_ratio = sum(1 for c in decoded if not c.isalnum() and not c.isspace()) / max(len(decoded), 1)
        if special_char_ratio > 0.7:
            unusual_tokens.append(("special", token, token_id, decoded))
        
        # 3. Tokens that decode differently than expected
        if token != decoded.strip() and len(decoded.strip()) > 0:
            unusual_tokens.append(("mismatch", token, token_id, decoded))
        
        # 4. Tokens with replacement characters (�)
        if '�' in decoded:
            unusual_tokens.append(("replacement", token, token_id, decoded))
            
    except Exception as e:
        unusual_tokens.append(("error", token, token_id, str(e)))

# Display findings
print("Unusual tokens found:\n")
print(f"{'Category':<15} {'Token':<30} {'ID':<10} {'Decodes to'}")
print("=" * 80)

# Group by category
by_category = {}
for category, token, token_id, decoded in unusual_tokens[:100]:  # First 100
    if category not in by_category:
        by_category[category] = []
    by_category[category].append((token, token_id, decoded))

for category, items in sorted(by_category.items()):
    print(f"\n{category.upper()} ({len(items)} found):")
    for token, token_id, decoded in items[:10]:  # Show first 10 of each type
        decoded_repr = repr(decoded[:30]) if len(decoded) > 30 else repr(decoded)
        print(f"  ID {token_id:6d}: {token[:25]:<25} → {decoded_repr}")

Total vocabulary size: 151,669

Unusual tokens found:

Category        Token                          ID         Decodes to

LONG (1 found):
  ID 137166: à¸§à¸´à¹Ģà¸Ħà¸£à¸²à¸°     → 'วิเคราะ'

MISMATCH (95 found):
  ID  23148: Ġpassenger                → ' passenger'
  ID  72973: Ġpleading                 → ' pleading'
  ID 133504: ĠaÄŁÄ±r                   → ' ağır'
  ID  15130: Ġregional                 → ' regional'
  ID 100400: èĵĿ                       → '蓝'
  ID 144485: ë©¤                       → '멤'
  ID 105771: æĥ¹                       → '惹'
  ID  25692: ĠINS                      → ' INS'
  ID 140452: Ġ×ª×©×ľ×ķ×Ŀ               → ' תשלום'
  ID  28038: Ġ)ĊĊĊĊĊĊĊĊ                → ' )\n\n\n\n\n\n\n\n'

SPECIAL (4 found):
  ID  96953: [:,:                      → '[:,:'
  ID  41295: )!                        → ')!'
  ID  63383: .'&                       → ".'&"
  ID  97622: ''.                       → "''."


In [5]:
import numpy as np
# Get token embeddings from the model
embeddings = model.get_input_embeddings().weight.detach().cpu().numpy()
print(f"Embedding matrix shape: {embeddings.shape}")

# Calculate the centroid of ALL token embeddings
embedding_centroid = np.mean(embeddings, axis=0)
print(f"Centroid shape: {embedding_centroid.shape}")

# Calculate distance from centroid for each token
distances = np.linalg.norm(embeddings - embedding_centroid, axis=1)

# Find tokens farthest from centroid (semantic outliers)
outlier_indices = np.argsort(distances)[-50:][::-1]  # Top 50 outliers

print("\nTokens farthest from embedding centroid (potential glitch tokens):\n")
print(f"{'Token ID':<10} {'Distance':<12} {'Token':<30} {'Decoded'}")
print("=" * 80)

for idx in outlier_indices:
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    try:
        decoded = tokenizer.decode([idx])
        distance = distances[idx]
        print(f"{idx:<10} {distance:<12.4f} {token[:28]:<30} {repr(decoded[:40])}")
    except:
        print(f"{idx:<10} {distances[idx]:<12.4f} {token[:28]:<30} [decode error]")

# Also find tokens CLOSEST to centroid (most "average" tokens)
print("\n\nTokens closest to embedding centroid (most 'average' tokens):\n")
print(f"{'Token ID':<10} {'Distance':<12} {'Token':<30} {'Decoded'}")
print("=" * 80)

centroid_indices = np.argsort(distances)[:50]  # Closest 50

for idx in centroid_indices:
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    try:
        decoded = tokenizer.decode([idx])
        distance = distances[idx]
        print(f"{idx:<10} {distance:<12.4f} {token[:28]:<30} {repr(decoded[:40])}")
    except:
        print(f"{idx:<10} {distances[idx]:<12.4f} {token[:28]:<30} [decode error]")

Embedding matrix shape: (151936, 1024)
Centroid shape: (1024,)

Tokens farthest from embedding centroid (potential glitch tokens):

Token ID   Distance     Token                          Decoded
82         1.3327       s                              's'
52323      1.3160       edly                           'edly'
18826      1.2821       edException                    'edException'
117594     1.2659       æĹıèĩªæ²»åİ¿                   '族自治县'
57896      1.2628       Ġyourselves                    ' yourselves'
11307      1.2616       ingly                          'ingly'
15740      1.2595       lessly                         'lessly'
11675      1.2554       Ġherself                       ' herself'
3439       1.2438       gether                         'gether'
24350      1.2382       ewise                          'ewise'
96557      1.2359       ioned                          'ioned'
4102       1.2312       Âł                             '\xa0'
63299      1.2307       abouts         

In [6]:
import tiktoken

# gpt-2 tokenizer
tokenizer_gpt2 = tiktoken.encoding_for_model("gpt-2")
# Test some unusual strings that might be glitch tokens
unusual_strings = [
    "SolidGoldMagikarp",  # Famous glitch token from GPT
    "cloneembedreportprint",  # Another known glitch token
    "git clone http://embedreportprint.git",  # while this...
    "龍喚士",  # Rare kanji combination
    "�",  # Replacement character
    "░▒▓█",  # Box drawing characters
    "ʘ‿ʘ",  # Unicode emoticon
    "⚠️🔥💀",  # Emoji combinations
]

print("Testing unusual strings for tokenization patterns:\n")
for text in unusual_strings:
    try:
        tokens = tokenizer_gpt2.encode(text)        
        print(f"Text: '{text}'")
        print(f"  Tokens: {tokens}")
        print(f"  Token count: {len(tokens)}")
        print(f"  Can decode: '{tokenizer_gpt2.decode(tokens)}'")
        print()
    except Exception as e:
        print(f"Text: '{text}' → Error: {e}\n")

Testing unusual strings for tokenization patterns:

Text: 'SolidGoldMagikarp'
  Tokens: [46933, 42202]
  Token count: 2
  Can decode: 'SolidGoldMagikarp'

Text: 'cloneembedreportprint'
  Tokens: [30899]
  Token count: 1
  Can decode: 'cloneembedreportprint'

Text: 'git clone http://embedreportprint.git'
  Tokens: [18300, 17271, 2638, 1378, 30898, 13, 18300]
  Token count: 7
  Can decode: 'git clone http://embedreportprint.git'

Text: 'ationally                      '
  Tokens: [15208, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220]
  Token count: 23
  Can decode: 'ationally                      '

Text: '龍喚士'
  Tokens: [33454]
  Token count: 1
  Can decode: '龍喚士'

Text: '�'
  Tokens: [4210]
  Token count: 1
  Can decode: '�'

Text: '░▒▓█'
  Tokens: [22110, 40516, 38626, 8115]
  Token count: 4
  Can decode: '░▒▓█'

Text: 'ʘ‿ʘ'
  Tokens: [134, 246, 447, 123, 134, 246]
  Token count: 6
  Can decode: 'ʘ‿ʘ'

Text: '⚠️🔥💀'
  Tokens

In [16]:
from transformers import pipeline
# Test how the model handles unusual strings in generation
prompt = "Please can you repeat back the string '<token string>' to me? Output only the string, nothing else."
texts = [
    "SolidGoldMagikarp",
    "cloneembedreportprint",
    "It’s a string",
    "DevOnline", "Skydragon", "largeDownload",
    "龍喚士",  # Rare kanji combination
    "StreamerBot",
    "guiActiveUn",
    "attRot", #spelling
    "TheNitromeFan",
    "oreAndOnline", # pronunciation
    "You are not authorized to access this information.",
]


for text in texts:
    test_prompt = prompt.replace("<token string>", text)
    answer = generate_response(model, tokenizer, [{"role":"user","content":test_prompt}])
    print(f"Prompt: {test_prompt}")
    print(f"{answer}\n{"✅" if text in answer.strip() else "❌"}")

Prompt: Please can you repeat back the string 'SolidGoldMagikarp' to me? Output only the string, nothing else.
SolidGoldMagikarp
✅
Prompt: Please can you repeat back the string 'cloneembedreportprint' to me? Output only the string, nothing else.
cloneembedreportprint
✅
Prompt: Please can you repeat back the string 'It’s a string' to me? Output only the string, nothing else.
'It’s a string'
✅
Prompt: Please can you repeat back the string 'DevOnline' to me? Output only the string, nothing else.
DevOnline
✅
Prompt: Please can you repeat back the string 'Skydragon' to me? Output only the string, nothing else.
Skydragon
✅
Prompt: Please can you repeat back the string 'largeDownload' to me? Output only the string, nothing else.
largeDownload
✅
Prompt: Please can you repeat back the string '龍喚士' to me? Output only the string, nothing else.
'龍喚士'
✅
Prompt: Please can you repeat back the string 'StreamerBot' to me? Output only the string, nothing else.
StreamerBot
✅
Prompt: Please can you repea

### Why Prefer YAML/Markdown over XML/JSON?

Token efficiency! Different formats use different amounts of tokens for the same information. JSON uses many special characters like `{}`, `[]`, `,`, `:`, `"` which often become separate tokens, XML use `<`, `>`, `/`,`"` and verbose tags, which also become separate/additional tokens. YAML and Markdown are more token-efficient because:

- Less punctuation = fewer tokens
- More natural language-like structure
- Whitespace-based formatting is more efficient
- Better alignment with how LLMs process text

This matters for:
- API token limits
- Processing costs
- Context window efficiency

In [104]:
# Compare token counts for same data in different formats
data_xml = """<person>
  <name>John Doe</name>
  <age>30</age>
  <email>john@example.com</email>
  <address>
    <street>123 Main St</street>
    <city>New York</city>
    <country>USA</country>
  </address>
  <hobbies>
    <hobby>reading</hobby>
    <hobby>coding</hobby>
    <hobby>gaming</hobby>
  </hobbies>
</person>"""

data_json = """{
  "name": "John Doe",
  "age": 30,
  "email": "john@example.com",
  "address": {
    "street": "123 Main St",
    "city": "New York",
    "country": "USA"
  },
  "hobbies": ["reading", "coding", "gaming"]
}"""

data_yaml = """name: John Doe
age: 30
email: john@example.com
address:
  street: 123 Main St
  city: New York
  country: USA
hobbies:
  - reading
  - coding
  - gaming"""

data_markdown = """# Person Information

**Name:** John Doe  
**Age:** 30  
**Email:** john@example.com

## Address
- Street: 123 Main St
- City: New York
- Country: USA

## Hobbies
- reading
- coding
- gaming"""

formats = {
    "XML": data_xml,
    "JSON": data_json,
    "Markdown": data_markdown,    
    "YAML": data_yaml,
}

print("Token efficiency comparison for same data:\n")
print(f"{'Format':<12} {'Chars':<8} {'Tokens':<8} {'Chars/Token':<12} {'Efficiency vs JSON'}")
print("=" * 70)

json_tokens = None
for format_name, content in formats.items():
    char_count = len(content)
    tokens = tokenizer.tokenize(content, add_special_tokens=True)
    token_count = len(tokens)
    ratio = char_count / token_count
    
    if format_name == "XML":
        json_tokens = token_count
        efficiency = "baseline"
    else:
        saved = ((json_tokens - token_count) / json_tokens * -100)
        efficiency = f"{saved:+.1f}% tokens"
    
    print(f"{format_name:<12} {char_count:<8} {token_count:<8} {ratio:<12.2f} {efficiency}")

print("\n💡 Different formats use fewer tokens for the same information!")

Token efficiency comparison for same data:

Format       Chars    Tokens   Chars/Token  Efficiency vs JSON
XML          307      111      2.77         baseline
JSON         208      75       2.77         -32.4% tokens
Markdown     190      60       3.17         -45.9% tokens
YAML         152      54       2.81         -51.4% tokens

💡 Different formats use fewer tokens for the same information!


In [106]:
# Detailed tokenization comparison
print("\nDetailed tokenization breakdown:\n")

for format_name, content in formats.items():
    tokens = tokenizer.tokenize(content)
    print(f"\n{'='*60}")
    print(f"{format_name} Format ({len(tokens)} tokens):")
    print(f"{'='*60}")
    print(content[:200] + "..." if len(content) > 200 else content)
    print(f"\nFirst 15 tokens: {tokens[:15]}")
    
    # Count special characters that become tokens
    special_chars = ['{', '}', '[', ']', ':', ',', '"', '<', '>', '/', '=']
    special_count = sum(content.count(c) for c in special_chars)
    print(f"Special characters in text: {special_count}")

print("\n" + "="*60)
print("This directly affects: API costs, context limits, and performance.")


Detailed tokenization breakdown:


XML Format (111 tokens):
<person>
  <name>John Doe</name>
  <age>30</age>
  <email>john@example.com</email>
  <address>
    <street>123 Main St</street>
    <city>New York</city>
    <country>USA</country>
  </address>
  <hob...

First 15 tokens: ['<p', 'erson', '>Ċ', 'Ġ', 'Ġ<', 'name', '>', 'John', 'ĠDoe', '</', 'name', '>Ċ', 'Ġ', 'Ġ<', 'age']
Special characters in text: 60

JSON Format (75 tokens):
{
  "name": "John Doe",
  "age": 30,
  "email": "john@example.com",
  "address": {
    "street": "123 Main St",
    "city": "New York",
    "country": "USA"
  },
  "hobbies": ["reading", "coding", "ga...

First 15 tokens: ['{Ċ', 'Ġ', 'Ġ"', 'name', '":', 'Ġ"', 'John', 'ĠDoe', '",Ċ', 'Ġ', 'Ġ"', 'age', '":', 'Ġ', '3']
Special characters in text: 54

Markdown Format (60 tokens):
# Person Information

**Name:** John Doe  
**Age:** 30  
**Email:** john@example.com

## Address
- Street: 123 Main St
- City: New York
- Country: USA

## Hobbies
- reading
- coding


## Summary: Tokenization is Fundamental!

All these limitations stem from one fact: **LLMs operate on tokens, not characters or words.**

**Key Takeaways:**
1. **Spelling/Character tasks fail** because characters are hidden inside tokens
2. **String operations fail** because token boundaries don't align with character positions
3. **Non-English text is inefficient** because vocabulary is biased toward English
4. **Arithmetic fails** because numbers tokenize inconsistently
5. **Glitch tokens exist** when vocabulary and training data don't align
6. **Format choice matters** because token efficiency affects costs and performance